# Tutorial

In [1]:
1

1

In [2]:
import os
import scanpy as sc
import pickle
import sys
from collections import Counter
sys.path.append("../..")

In [3]:
from idtrack import *
from idtrack._track_tests import *

In [4]:
logger_config()
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"  # or any other local directory
dm = DatabaseManager("homo_sapiens", 107, "gene", local_dir, 79) 

In [5]:
tt: TrackTests = TrackTests(dm)

2022-08-15 19:05:48 INFO:graph: The graph is being read.


In [6]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Meyer_2021": [f"{dset0_dir}/meyer_2021.h5ad"],
    "MeyerNikolic_unpubl": [f"{dset0_dir}/meyer_nikolic_unpubl.h5ad"],
    "Barbry_unpubl": [f"{dset0_dir}/barbry.h5ad"],
    "Regev_2021": [
        f"{dset0_dir}/delorey_cryo.h5ad", f"{dset0_dir}/delorey_fresh.h5ad",
        f"{dset0_dir}/delorey_nuclei.h5ad"
    ],
    "Thienpont_2018": [f"{dset1_dir}/Lambrechts/lambrechts.h5ad"],
    "Budinger_2020": [f"{dset0_dir}/bharat.h5ad"],
    "Banovich_Kropski_2020": [f"{dset0_dir}/haberman.h5ad"],
    "Sheppard_2020": [f"{dset0_dir}/tsukui.h5ad"],
    "Wunderink_2021": [
        f"{dset0_dir}/grant_cryo.h5ad", f"{dset0_dir}/grant_fresh.h5ad"
    ],
    "Lambrechts_2021": [
        f"{dset0_dir}/wouters.h5ad" #, f"{dset0_dir}/wouters_labs.h5ad"
    ],
    "Zhang_2021": [f"{dset1_dir}/Liao/covid_for_publish.h5ad"],
    "Duong_lungMAP_unpubl": [f"{dset0_dir}/duong.h5ad"],
    "Janssen_2020": [f"{dset0_dir}/mould.h5ad"],
    "Sun_2020": [
        f"{dset0_dir}/wang_sub_batch1.h5ad", f"{dset0_dir}/wang_sub_batch2.h5ad",
        f"{dset0_dir}/wang_sub_batch3.h5ad", f"{dset0_dir}/wang_sub_batch4.h5ad"],
    "Gomperts_2021": [
        f"{dset0_dir}/carraro_ucla.h5ad", f"{dset0_dir}/carraro_cff.h5ad",
        f"{dset0_dir}/carraro_csmc.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Schiller_2020": [f"{dset0_dir}/mayr.h5ad"],
    "Misharin_Budinger_2018": [f"{dset0_dir}/reyfman_disease.h5ad"],
    "Shalek_2018": [f"{dset0_dir}/ordovasmontanes.h5ad"],
    "Schiller_2021": [f"{dset0_dir}/schiller_discovair.h5ad"],
    "Peer_Massague_2020": [f"{dset0_dir}/laughney.h5ad"],
    "Lafyatis_2019": [f"{dset0_dir}/valenzi.h5ad"],
    "Tata_unpubl": [f"{dset0_dir}/tata_unpubl.h5ad"],
    "Xu_2020": [f"{dset0_dir}/guo.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"]
}

adata_dict = {
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
}

In [7]:
result = dict()
for ddadaata in adata_dict:
    
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))
    
    gene_list = set(gene_list)
    print(ddadaata, len(gene_list_filtered), len(gene_list_filtered)-len(set(gene_list_filtered)))
    gene_list_filtered = set(gene_list_filtered)
   
    for i in gene_list_filtered:
        if i not in tt.graph.nodes:
            raise ValueError
    print(tt.identify_source(gene_list_filtered)[0])
    
    a = dict()
    for ind, i in enumerate(gene_list_filtered):
        if ind % 100 == 0 or ind > len(gene_list_filtered)-5:
            progress_bar(ind, len(gene_list_filtered)-1)
        conv = tt.convert(i, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=True)
        a[i] = conv
    result[ddadaata] = a
    print(list(Counter([len(a[i]) if a[i] is not None else 0 for i in a]).most_common())[:10])
    print()

ValueError: 

In [6]:
with open(os.path.join(local_dir, "results_databases.pk"), 'rb') as handle:
    result = pickle.load(handle)
    print("Loaded")

Loaded


In [27]:
### Check how many total targets are 

In [7]:
only_matches = {ddad: {k: list(i.keys()) if i is not None else [] for k, i in result[ddad].items()} for ddad in adata_dict}
all_targets = {k for _, i in only_matches.items() for _, j in i.items() for k in j}
all_targets = sorted(all_targets)
len(all_targets)

56508

In [8]:
for ddd in only_matches:
    db_1to1 = [l for c, t in only_matches[ddd].items() for l in t if len(t) == 1]
    print(ddd, '\t','\t', len(db_1to1), len(set(db_1to1)), len(db_1to1)-len(set(db_1to1)))

Kaminski_2020 	 	 45377 44670 707
Meyer_2021 	 	 20708 20183 525
MeyerNikolic_unpubl 	 	 32429 31490 939
Barbry_unpubl 	 	 16579 16258 321
Regev_2021 	 	 29980 29199 781
Thienpont_2018 	 	 27403 26836 567
Budinger_2020 	 	 25564 24939 625
Banovich_Kropski_2020 	 	 32825 32105 720
Sheppard_2020 	 	 26587 26039 548
Wunderink_2021 	 	 21280 20794 486
Lambrechts_2021 	 	 32429 31490 939
Zhang_2021 	 	 18326 17885 441
Duong_lungMAP_unpubl 	 	 26853 26210 643
Janssen_2020 	 	 32429 31490 939
Sun_2020 	 	 25788 25189 599
Gomperts_2021 	 	 30077 25040 5037
Eils_2020 	 	 31505 30599 906
Schiller_2020 	 	 30913 27335 3578
Misharin_Budinger_2018 	 	 26612 26057 555
Shalek_2018 	 	 24519 23946 573
Schiller_2021 	 	 17368 16937 431
Peer_Massague_2020 	 	 18894 18453 441
Lafyatis_2019 	 	 21921 21304 617
Tata_unpubl 	 	 31073 30229 844
Xu_2020 	 	 31504 30598 906
Sims_2019 	 	 55250 53984 1266
Schultze_unpubl 	 	 23895 23397 498


In [8]:
klk = [i for i in db_1to1 if db_1to1.count(i) > 1]
np.array(klk[:10])

array(['ENSG00000006042.12', 'ENSG00000104643.10', 'ENSG00000013573.17',
       'ENSG00000196734.9', 'ENSG00000132207.18', 'ENSG00000129071.10',
       'ENSG00000015568.13', 'ENSG00000169710.9', 'ENSG00000231924.10',
       'ENSG00000159131.17'], dtype='<U18')

In [ ]:
print("Worked flawlessly")

In [26]:
for ddadaata in result:
    #ddadaata = "Tata_unpubl"
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))
    print(gene_list_lost[:10])

[]
[]


2022-08-15 16:44:03 WARNING:track: Number of converted IDs with small modifications: 18
2022-08-15 16:44:03 WARNING:track: Number of IDs not found in the graph: 50


['ABCF2-1', 'ATXN7-1', 'CCDC39-1', 'COG8-1', 'CYB561D2-1', 'DIABLO-1', 'EMG1-1', 'H2BFS-1', 'HSPA14-1', 'IGF2-1']


2022-08-15 16:44:20 WARNING:track: Number of IDs not found in the graph: 7


['TBCE-1', 'MATR3-1', 'TMSB15B-1', 'HSPA14-1', 'GOLGA8M-1', 'CYB561D2-1', 'CDC20Bshort']


...

['U1-1', 'EFNA3-1', 'DCAF8-1', 'MRPL30-1', 'LIMS3-1', 'KRBOX1-1', 'TLR9-1', 'RPP14-1', 'CRYBG3-1', 'TM4SF19-1']


2022-08-15 16:45:25 WARNING:track: Number of IDs not found in the graph: 1881


['U3-1', 'Y_RNA-1', 'Y_RNA-2', 'SCARNA17-1', 'SNORA40-1', 'Metazoa_SRP-1', 'Y_RNA-3', 'Metazoa_SRP-2', 'Y_RNA-4', 'Metazoa_SRP-3']


2022-08-15 16:45:25 WARNING:track: Number of converted IDs with small modifications: 1


[]


In [44]:
'SCARNA17' in tt.graph.nodes

True

In [45]:
ddadaata = "Sims_2019"
adata = sc.read(adata_dict[ddadaata][0])
gene_list = list(adata.var.index)
gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))
print(gene_list_lost[:10])

2022-08-15 17:05:15 WARNING:track: Number of IDs not found in the graph: 1881


['U3-1', 'Y_RNA-1', 'Y_RNA-2', 'SCARNA17-1', 'SNORA40-1', 'Metazoa_SRP-1', 'Y_RNA-3', 'Metazoa_SRP-2', 'Y_RNA-4', 'Metazoa_SRP-3']


In [68]:
"SCARNA17-2" in tt.graph.nodes

False

In [147]:
l=[i for i in result[ddadaata] if result[ddadaata][i] is None]
len(l)

3465

In [151]:
tt.convert(l[1], 107, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=True)

{'ENSG00000249406.3': [-1, -1, nan, 1, 0, 11]}

In [ ]:
ddad = "Schultze_unpubl"
only_matches = {ddad: {k: list(i.keys()) if i is not None else [] for k, i in result[ddad].items()}}
for ddd in only_matches:
    db_1to1 = [l for c, t in only_matches[ddd].items() for l in t if len(t) == 1]
    print(ddd, '\t','\t', len(db_1to1), len(set(db_1to1)), len(db_1to1)-len(set(db_1to1)))
klk = [i for i in db_1to1 if db_1to1.count(i) > 1]
np.array(klk[:10])

In [54]:
tt.convert("POLR1A", None, None, 'ensembl_gene', return_path=True, prioritize_to_one_filter=False)

{'ENSG00000068654.17': [1,
  1,
  0.9793575999999999,
  5,
  (('POLR1A', 'ENSG00000068654.14', 0, ('ensembl_gene', 80)),
   ('ENSG00000068654.14', 'ENSG00000068654.15', 0),
   ('ENSG00000068654.15', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 1),
   ('ENSG00000068654.16', 'ENSG00000068654.17', 0))],
 'ENSG00000132383.12': [1,
  3,
  0.9870546666666667,
  3,
  (('POLR1A', 'ENSG00000068654.11', 0, ('assembly_37_ensembl_gene', 80)),
   ('ENSG00000068654.11', 'RPA1', 0, ('external', 80)),
   ('RPA1', 'ENSG00000132383.10', 0, ('ensembl_gene', 80)),
   ('ENSG00000132383.10', 'ENSG00000132383.11', 0),
   ('ENSG00000132383.11', 'ENSG00000132383.12', 0),
   ('ENSG00000132383.12', 'ENSG00000132383.12', 0))]}

In [55]:
tt.convert("RPA1", None, None, 'ensembl_gene', return_path=True, prioritize_to_one_filter=False)

{'ENSG00000132383.12': [1,
  1,
  0.9870546666666667,
  3,
  (('RPA1', 'ENSG00000132383.10', 0, ('ensembl_gene', 80)),
   ('ENSG00000132383.10', 'ENSG00000132383.11', 0),
   ('ENSG00000132383.11', 'ENSG00000132383.12', 0),
   ('ENSG00000132383.12', 'ENSG00000132383.12', 0))],
 'ENSG00000068654.17': [1,
  1,
  0.9793575999999999,
  5,
  (('RPA1', 'ENSG00000068654.14', 0, ('ensembl_gene', 80)),
   ('ENSG00000068654.14', 'ENSG00000068654.15', 0),
   ('ENSG00000068654.15', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 1),
   ('ENSG00000068654.16', 'ENSG00000068654.17', 0))]}

In [64]:
all_paths: set = set()
tt._recursive_path_search("PAK6", 107, 107, all_paths, False, DB.external_search_settings, external_jump=np.inf)
all_paths

{(('PAK6', 'ENSG00000137843.12', 0, ('ensembl_gene', 97)),),
 (('PAK6', 'ENSG00000137843.7', 0, ('assembly_37_ensembl_gene', 102)),
  ('ENSG00000137843.7', 'PAK5', 0, ('external', 102)),
  ('PAK5', 'ENSG00000101349.17', 0, ('ensembl_gene', 102)))}

In [53]:
tt.convert("POLR1A", 79, None, 'ensembl_gene', return_path=True, prioritize_to_one_filter=False)

{'ENSG00000068654.17': [1,
  1,
  0.9793575999999999,
  5,
  (('POLR1A', 'ENSG00000068654.14', 0, ('ensembl_gene', 80)),
   ('ENSG00000068654.14', 'ENSG00000068654.15', 0),
   ('ENSG00000068654.15', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 0),
   ('ENSG00000068654.16', 'ENSG00000068654.16', 1),
   ('ENSG00000068654.16', 'ENSG00000068654.17', 0))],
 'ENSG00000132383.12': [1,
  3,
  0.9870546666666667,
  3,
  (('POLR1A', 'ENSG00000068654.11', 0, ('assembly_37_ensembl_gene', 80)),
   ('ENSG00000068654.11', 'RPA1', 0, ('external', 80)),
   ('RPA1', 'ENSG00000132383.10', 0, ('ensembl_gene', 80)),
   ('ENSG00000132383.10', 'ENSG00000132383.11', 0),
   ('ENSG00000132383.11', 'ENSG00000132383.12', 0),
   ('ENSG00000132383.12', 'ENSG00000132383.12', 0))]}

In [75]:
tt.convert("PAK5", 79, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=False)
## bu neden 1 varken 2'yi de aldi amk

{'ENSG00000137843.12': [1, 1, 0.99267325, 4],
 'ENSG00000259288.7': [1, 1, 0.6713326666666667, 3],
 'ENSG00000101349.17': [1, 1, 0.9972175, 2]}

In [71]:
tt.convert("PAK6", 79, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=False)
## bu neden 1 varken 2'yi de aldi amk

{'ENSG00000259288.7': [1, 1, 0.6713326666666667, 3],
 'ENSG00000101349.17': [1, 3, 0.9972175, 2],
 'ENSG00000137843.12': [1, 1, 0.99267325, 4]}

In [31]:
tt.should_graph_reversed("PAK6", 107)

('forward', 107)

In [47]:
tt.get_active_ranges_of_id_others("PAK6")

[[79, 107]]

In [14]:
for jjj in range(20,40):
    lll = [(c,t) for c, t in only_matches[ddd].items() if klk[jjj] in t]
    for llll in [i for i, j in lll]:
        print(tt.convert(llll, None, None, 'ensembl_gene', return_path=True, prioritize_to_one_filter=True))
    print()

{'ENSG00000010810.17': [-1, -1, 7, 0, 29, (('FYN', 'ENSG00000010810.17', 0, ('ensembl_gene', 81)),)]}
{'ENSG00000010810.17': [-1, -3, 7, 0, 29, (('AC036108.1', 'ENSG00000269283.1', 0, ('assembly_37_ensembl_gene', 81)), ('ENSG00000269283.1', 'SYN', 0, ('external', 81)), ('SYN', 'ENSG00000010810.17', 0, ('ensembl_gene', 81)))]}
{'ENSG00000010810.17': [-1, -1, 7, 0, 29, (('SLK', 'ENSG00000010810.17', 0, ('ensembl_gene', 81)),)]}

{'ENSG00000233694.6': [-1, -1, 3, 0, 2, (('AC007365.1', 'ENSG00000233694.6', 0, ('ensembl_gene', 97)),)]}
{'ENSG00000233694.6': [-1, -1, 3, 0, 2, (('LINC02579', 'ENSG00000233694.6', 0, ('ensembl_gene', 97)),)]}

{'ENSG00000250751.1': [-1, -1, 4, 0, 1, (('AC015795.1', 'ENSG00000250751.1', 0, ('ensembl_gene', 79)),)]}
{'ENSG00000250751.1': [-1, -1, 4, 0, 1, (('AC015795.2', 'ENSG00000250751.1', 0, ('ensembl_gene', 79)),)]}

{'ENSG00000108684.15': [-1, -1, 9, 0, 5, (('ASIC2', 'ENSG00000108684.15', 0, ('ensembl_gene', 102)),)]}
{'ENSG00000108684.15': [-1, -1, 9, 0, 5,

In [15]:
rc37 = Dataset(dm.change_server(37), narrow_search=False).initialize_external_conversion()
#rc38 = Dataset(dm.change_server(38), narrow_search=False).initialize_external_conversion()

2022-08-12 22:26:23 INFO:dataset: Comparison data frame is being constructed for releases: [107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79].


In [16]:
d=np.unique(rc37['id_db'])

In [17]:
[i for i in d if "YX65C7" in i]

['XXyac-YX65C7_A.2', 'XXyac-YX65C7_A.3', 'XXyac-YX65C7_A.4']

In [18]:
"PRED58" in d

True

In [19]:
amk = [i for i in a if i not in d]

In [20]:
print(len([i for i in a if i in d]))
print(len(amk))

58
113


In [21]:
np.array([i for i in amk if i.split('.')[0] not in d])

array(['RP11-442N24--B.1', 'RP11-99J16--A.2', 'RP11-59D5--B.2',
       'RP11-445L13--B.3', 'RP11-544L8--B.4', 'XXyac-YX65C7-A.2',
       'XXyac-YX65C7-A.3', 'RP11-524D16--A.3', 'RP11-453F18--B.1',
       'Metazoa-SRP', 'Y-RNA', 'XX-DJ76P10--A.2', '5S-rRNA', 'Y-RNA.1',
       'Y-RNA.2', 'RP11-1157N2--B.2', 'RP1-213J1P--B.1',
       'RP1-213J1P--B.2', 'RP4-633O19--A.1', 'RP4-754E20--A.5',
       'CTA-280A3--B.2'], dtype='<U16')

In [22]:
# sonra da combinatorial olarak _ to - degisikligi yap..
 
# np.array([i for i in amk if i.split('.')[0] not in d])

In [23]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate([i for i in a if i in d]):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,

In [24]:
np.unique(rc37[rc37["id_db"] == 'FLJ27365']["name_db"])

array(['UniProtKB Gene Name', 'Uniprot_gn'], dtype=object)

In [26]:
ex=dm.create_external_all()

In [36]:
np.unique(ex[ex['assembly'] == 37]['name_db'])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene',
       'RFAM'], dtype=object)

array(['RFAM'], dtype=object)

In [57]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate(a):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [59]:
Counter([j for i in jkl for j in jkl[i]])

Counter({'Clone-based (Vega) gene': 377,
         'Clone_based_vega_gene': 377,
         'Clone-based (Vega)': 377,
         'Clone-based (Ensembl) gene': 87,
         'Clone-based (Ensembl)': 87,
         'Clone_based_ensembl_gene': 87,
         'EntrezGene': 41,
         'Uniprot_gn': 42,
         'HGNC Symbol': 20,
         'NCBI gene (formerly Entrezgene)': 41,
         'WikiGene': 17,
         'UniProtKB Gene Name': 42,
         'RFAM': 2})

In [6]:
df=dm.create_external_all()

In [5]:
ExternalDatabases(dm).give_list_for_case(give_type="db")

['EntrezGene',
 'UniProtKB Gene Name',
 'NCBI gene (formerly Entrezgene)',
 'HGNC Symbol']

In [8]:
np.sum(df['assembly']==38)

202607

In [9]:
np.sum(df['assembly']==37)

76298

In [13]:
np.unique(df[df['assembly']==37]["name_db"])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene'],
      dtype=object)

In [12]:
df[df['graph_id']=='ENSG00000173727.7']

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity,assembly
202611,107,ENSG00000173727.7,AP000769.1,Clone_based_ensembl_gene,NaN,NaN,37


In [110]:
the_dict[db_name]["Assembly"]

{'37': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False},
 '38': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False}}

In [3]:

self = ExternalDatabases(dm)
give_type='assembly'
the_dict_loaded = self.load_modified_yaml()
the_dict = the_dict_loaded[self.db_manager.organism][self.db_manager.form]

result = set()
for db_name in the_dict:
    for asm in the_dict[db_name]["Assembly"]:
        item = the_dict[db_name]["Assembly"][asm]
        res_ens = map(int, item["Ensembl release"].split(","))
        if self.db_manager.ensembl_release in res_ens and item["Include"]:
            if give_type == "db" and int(asm) == self.db_manager.ensembl_mysql_server:
                result.add(db_name)
            elif give_type == "assembly":
                result.add(int(asm))
            else:
                raise ValueError
            
            
result

{37, 38}

In [7]:
item

{'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
 'Include': False}

In [6]:

for i in dm.available_releases:
    ex = ExternalDatabases(dm.change_release(i))
    ass = ex.give_list_for_case(give_type="db")
    print(f"Rel:{i}, Ass{ass}")

Rel:79, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:80, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:81, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:82, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:83, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:84, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:85, Ass['EntrezGene', 'Vega gene', 'Clone_based_

In [68]:
kalds=[i for i in jkl if len(jkl[i]) == 0]
len(kalds)

113

In [37]:
bok = rc37[rc37["id_db"].isin(a)]

In [35]:
dsd = []
for i in a:
    c =set([j for j in np.unique(bok[bok["id_db"] == i]["name_db"])])
    #print(c)
    dsd.extend(c)
from collections import Counter
Counter(dsd).most_common()

[('Clone-based (Vega) gene', 377),
 ('Clone_based_vega_gene', 377),
 ('Clone-based (Vega)', 377),
 ('Clone-based (Ensembl) gene', 87),
 ('Clone-based (Ensembl)', 87),
 ('Clone_based_ensembl_gene', 87),
 ('Uniprot_gn', 42),
 ('UniProtKB Gene Name', 42),
 ('EntrezGene', 41),
 ('NCBI gene (formerly Entrezgene)', 41),
 ('HGNC Symbol', 20),
 ('WikiGene', 17),
 ('RFAM', 2)]